In [ ]:
from pathlib import Path
from tqdm import tqdm
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import transforms

from leaf.dta import LeafDataset, GetPatches, TransformPatches, RandomGreen, LeafDataLoader
from leaf.model import LeafModel, train_one_epoch, validate_one_epoch, warmup


# Transforms with normalizations for imagenet
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        RandomGreen(224, 224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'warmup': transforms.Compose([
        transforms.ToTensor(),
        RandomGreen(224, 224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'patches': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [ ]:
save_dir = Path("/mnt/hdd/leaf-disease-outputs")
save_dir.mkdir(exist_ok=True)
logging_dir = Path("/mnt/hdd/leaf-disease-runs")
logging_dir.mkdir(exist_ok=True)

batch_size = 16
val_batch_size = 32

num_workers = 4

learning_rate = 1e-4
#min_learning_rate = 1e-6
weight_decay = 1e-6
warmup_lr = 1e-7

#T_0 = 10

patches_logging_steps = 1000
save_checkpoints = True

train_dset = LeafDataset("./data/train_images", "./data/train_images/labels.csv", transform=data_transforms["train"])
patches_dset = LeafDataset("./data/patches_train", "./data/patches_train/labels.csv", extended_labels=True, transform=data_transforms["patches"])
warmup_dset = LeafDataset("./data/train_images", "./data/train_images/labels.csv", transform=data_transforms["warmup"])
val_dset = LeafDataset("./data/patches_val", "./data/patches_val/labels.csv", extended_labels=True, transform=data_transforms["val"])

train_dataloader = LeafDataLoader(train_dset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
patches_dataloader = LeafDataLoader(patches_dset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = LeafDataLoader(val_dset, batch_size=val_batch_size, shuffle=False, num_workers=num_workers)

leaf_model = LeafModel("tf_efficientnet_b4_ns", model_prefix="MONEVE", save_dir=save_dir, logging_dir=logging_dir)

# optimizer = Adam(leaf_model.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = Adam(leaf_model.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None
#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=T_0, T_mult=1, eta_min=min_learning_rate, last_epoch=-1)
leaf_model.update_optimizer_scheduler(optimizer, scheduler)

In [ ]:
warmup(leaf_model, train_dataloader, 500, warmup_lr, log_warmup=True)
train_one_epoch(leaf_model, patches_dataloader, log_steps=patches_logging_steps, val_data_loader=val_dataloader, save_at_log_steps=save_checkpoints, epoch_name=1)

In [ ]:
val_loss, val_acc = validate_one_epoch(leaf_model, val_dataloader)

In [ ]:
val_loss, val_acc

In [ ]:
leaf_model.save_checkpoint("MONEVE_epoch_1", optimizer, 1)

In [ ]:
leaf_model.optimizer.param_groups[0]['lr'] = 1e-5

In [ ]:
train_one_epoch(leaf_model, patches_dataloader, log_steps=patches_logging_steps, val_data_loader=val_dataloader, save_at_log_steps=save_checkpoints, epoch_name=2)

In [ ]:
val_loss, val_acc = validate_one_epoch(leaf_model, val_dataloader)

In [ ]:
val_loss, val_acc

In [ ]:
leaf_model.save_checkpoint("MONEVE_epoch_2", optimizer, 2)

In [ ]:
leaf_model.optimizer.param_groups[0]['lr'] = 1e-5
leaf_model.load_checkpoint("MONEVE_epoch_2")

In [ ]:
leaf_model.model = leaf_model.model.to(leaf_model.device)

In [ ]:
val_loss, val_acc = validate_one_epoch(leaf_model, val_dataloader)

In [ ]:
val_loss, val_acc

In [ ]:
model = leaf_model.model
model.eval()

In [ ]:
np_original_fnames = np.array(val_dset.original_fnames)
np_fnames = np.array(val_dset.fnames)

In [ ]:
with torch.no_grad():
    logits_all = torch.zeros((val_dataloader.num_padded_samples, 5), dtype=float, device=leaf_model.device)
    labels_all = torch.zeros((val_dataloader.num_padded_samples), dtype=int, device=leaf_model.device)
    original_fnames = np.array([""] * val_dataloader.num_padded_samples, dtype=np.dtype("U22"))
    patch_idxs = np.zeros((val_dataloader.num_padded_samples,), dtype=int)
    i = 0
    for imgs, labels, idxs in tqdm(val_dataloader):
        imgs = imgs.to(leaf_model.device)
        labels = labels.to(leaf_model.device)
        bs = imgs.shape[0]
        logits_all[i:(i + bs), :] = leaf_model.model.forward(imgs)
        labels_all[i:(i + bs)] = labels
        original_fnames[i:(i + bs)] = np_original_fnames[idxs]
        patch_idxs[i:(i + bs)] = np.array([int(val_dset.fnames[idx][-7:][:3]) for idx in idxs])
        i += bs

In [ ]:
with torch.no_grad():    
    val_loss = leaf_model.loss_fn(logits_all, labels_all)
    preds_all = logits_all.argmax(axis=-1)
    val_acc = (labels_all == preds_all).sum().item() / i

In [ ]:
original_fnames[original_fnames == ""] = "-1.jpg"

In [ ]:
logits_all.cpu().numpy().shape

In [ ]:
pred_df = pd.DataFrame({
    "img_id": np.array([int(fname[:-4]) for fname in original_fnames]).astype(int),
    "patch_idx": patch_idxs.astype(int),
    "label": labels_all.cpu().numpy().astype(int),
    "logits_0": logits_all.cpu().numpy()[:,0],
    "logits_1": logits_all.cpu().numpy()[:,1],
    "logits_2": logits_all.cpu().numpy()[:,2],
    "logits_3": logits_all.cpu().numpy()[:,3],
    "logits_4": logits_all.cpu().numpy()[:,4],
    "pred": preds_all.cpu().numpy().astype(int)
})

In [ ]:
pred_df.sample(20)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def f1_df(label):
    return f1_score(pred_df["label"] == label, pred_df["pred"] == label)

# Mean logits

In [ ]:
img_df = pred_df.groupby("img_id").agg({"label": "first", "logits_0": "mean", "logits_1": "mean", "logits_2": "mean","logits_3": "mean", "logits_4": "mean"})

In [ ]:
img_df = img_df.loc[img_df.index != -1, :].reset_index()

In [ ]:
img_df

In [ ]:
img_preds = img_df.loc[:, ["logits_0", "logits_1", "logits_2", "logits_3", "logits_4"]].values.argmax(-1)

In [ ]:
img_df["pred"] = img_preds

In [ ]:
img_df = img_df.loc[:, ["img_id", "label", "pred"]]

In [ ]:
(img_df["label"] == img_df["pred"]).sum() / img_df.shape[0]

In [ ]:
only_234 = img_df.loc[img_df["label"].isin([2, 3, 4]), :]

In [ ]:
(only_234["label"] == only_234["pred"]).sum() / only_234.shape[0]

In [ ]:
only_0234 = img_df.loc[img_df["label"].isin([0, 2, 3, 4]), :]

In [ ]:
(only_0234["label"] == only_0234["pred"]).sum() / only_0234.shape[0]

# Variant

In [ ]:
def pred_to_one_hot(pred):
    vals = np.arange(5)
    assert pred in vals
    return (vals == pred).astype(int)

In [ ]:
pred_df_aug = pd.concat([pred_df, pd.DataFrame(np.array([pred_to_one_hot(pred) for pred in pred_df["pred"]]))], axis=1).rename(columns={i: f"pred_{i}" for i in range(5)})

In [ ]:
pred_df_aug = pred_df_aug.loc[pred_df["img_id"] != -1, :]

In [ ]:
pred_df_aug

In [ ]:
img_df = pred_df_aug.groupby("img_id").agg({**{"label": "first", "patch_idx": "max"},
                                        **{f"logits_{i}": "mean" for i in range(5)},
                                        **{f"pred_{i}": "sum" for i in range(5)}}).reset_index().rename(columns={"patch_idx": "n_patches"})

In [ ]:
img_df[img_df["label"] == 0].sample(20)

In [ ]:
img_df_mod = pd.concat([img_df,
           pd.DataFrame(np.apply_along_axis(softmax, 1, img_df.loc[:, ["logits_0", "logits_1", "logits_2", "logits_3", "logits_4"]].values)).rename(columns = {i: f"prob_{i}" for i in range(5)})], axis=1)

In [ ]:
img_df_mod.loc[img_df_mod["label"] == 1, :].sample(20)

In [ ]:
n_grid = 10
max_bonus = 0.1

In [ ]:
def get_acc(mod_probs):
    return (img_df_mod["label"].values == mod_probs.argmax(axis=1)).sum() / img_df.shape[0]

In [ ]:
get_acc(img_df_mod.loc[:, [f"prob_{i}" for i in range(5)]].values)

In [ ]:
#pbar = tqdm(total = max_bonus ** 5)
best_combo = None
best_acc = -1
for bonus_0 in np.linspace(0, max_bonus, n_grid+1):
    for bonus_1 in np.linspace(0, max_bonus, n_grid+1):
        for bonus_2 in np.linspace(0, max_bonus, n_grid+1):
            for bonus_3 in np.linspace(0, max_bonus, n_grid+1):
                for bonus_4 in np.linspace(0, max_bonus, n_grid+1):
                    mod_probs = img_df_mod.loc[:, [f"prob_{i}" for i in range(5)]].values
                    mod_probs[:, 0] += bonus_0
                    mod_probs[:, 1] += bonus_1
                    mod_probs[:, 2] += bonus_2
                    mod_probs[:, 3] += bonus_3
                    mod_probs[:, 4] += bonus_4
                    acc = get_acc(mod_probs)
                    if acc > best_acc:
                        best_acc = acc
                        best_combo = (bonus_0, bonus_1)
                    #pbar.update(1)
#pbar.close()

In [ ]:
best_acc

# Variant: SVM

In [ ]:
img_df_svm = img_df_mod.copy()

In [ ]:
img_df_svm["n_patches"] = img_df_svm["n_patches"] / img_df_svm["n_patches"].max()

In [ ]:
for i in range(5):
    img_df_svm[f"pred_{i}"] = img_df_svm[f"pred_{i}"] / 13

In [ ]:
img_df_svm = img_df_svm.drop(columns = [f"logits_{i}" for i in range(5)])

In [ ]:
from sklearn import svm

In [ ]:
X.shape

In [ ]:
X = img_df_svm.iloc[:, 2:].values

In [ ]:
y = img_df_svm["label"].values

In [ ]:
from random import shuffle

In [ ]:
idxs = list(range(X.shape[0]))

In [ ]:
shuffle(idxs)

In [ ]:
train_idxs = idxs[:2000]

In [ ]:
test_idxs = idxs[2000:]

In [ ]:
X_train, y_train = X[train_idxs, :], y[train_idxs]

In [ ]:
X_test, y_test = X[test_idxs, :], y[test_idxs]

In [ ]:
clf = svm.SVC(C=5, kernel="rbf")
clf.fit(X_train, y_train)

In [ ]:
np.sum((clf.predict(X_test) == y_test)) / X_test.shape[0]

In [ ]:
img_df_svm

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'multi:softmax', "num_class":5}
evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [ ]:
num_round = 1
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
y_pred = bst.predict(dtest)

In [ ]:
np.sum(y_pred == y_test) / len(y_test)

# Variant: softmax, then mean

In [ ]:
def softmax(x):
    return np.exp(x)/sum(np.exp(x))

In [ ]:
img_df = pd.concat([pred_df.loc[:, ["img_id", "patch_idx", "label"]],
           pd.DataFrame(np.apply_along_axis(softmax, 1, pred_df.loc[:, ["logits_0", "logits_1", "logits_2", "logits_3", "logits_4"]].values))], axis=1)
img_df.rename(columns = {i: f"prob_{i}" for i in range(5)}, inplace=True)

In [ ]:
img_preds = img_df.loc[:, [f"prob_{i}" for i in range(5)]].values.argmax(-1)

In [ ]:
img_df["pred"] = img_preds

In [ ]:
img_df = img_df.loc[:, ["img_id", "label", "pred"]]

In [ ]:
(img_df["label"] == img_df["pred"]).sum() / img_df.shape[0]

In [ ]:
np.apply_along_axis(softmax, 1, pred_df.loc[:, ["logits_0", "logits_1", "logits_2", "logits_3", "logits_4"]].values).shape

In [ ]:
img_df = pred_df.groupby("img_id").agg({"label": "first", "logits_0": "mean", "logits_1": "mean", "logits_2": "mean","logits_3": "mean", "logits_4": "mean",}).reset_index()

In [ ]:
f1_df(0), f1_df(1), f1_df(2), f1_df(3), f1_df(4)

In [ ]:
pred_df["label"].value_counts(normalize=True)

In [ ]:
f1_score(pred_df["label"] == 1normalize=ed_df["pred"] == 1)

In [ ]:
only_2 = pred_df.loc[pred_df["label"] == 2].copy()
(only_2["pred"] == 2).sum() / only_2.shape[0]

In [ ]:
only_1 = pred_df.loc[pred_df["label"] == 1].copy()

In [ ]:
(only_1["pred"] == 1).sum() / only_1.shape[0]

In [ ]:
only_3 = pred_df.loc[pred_df["label"] == 3].copy()
(only_3["pred"] == 3).sum() / only_3.shape[0]

In [ ]:
only_4 = pred_df.loc[pred_df["label"] == 4].copy()
(only_4["pred"] == 4).sum() / only_4.shape[0]

In [ ]:
only_0 = pred_df.loc[pred_df["label"] == 0].copy()
(only_0["pred"] == 0).sum() / only_0.shape[0]

In [ ]:
pred_df

In [ ]:
original_fnames[-33:]

In [ ]:
original_fnames

In [ ]:
'1003218714.jpg-002.jpg'[:-4]

In [ ]:
val_dset.fnames[2]

In [ ]:
val_dset.original_fnames[2]

In [ ]:
        imgs = imgs.to(leaf_model.device)
        labels = labels.to(leaf_model.device)
        bs = imgs.shape[0]
        logits_all[i:(i + bs), :] = leaf_model.model.forward(imgs)
        labels_all[i:(i + bs)] = labels
        i += bs

    val_loss = leaf_model.loss_fn(logits_all, labels_all)
    preds_all = logits_all.argmax(axis=-1)
    val_acc = (labels_all == preds_all).sum().item() / i

    return val_loss, val_acc

Not run:

In [ ]:
val_loss, val_acc = validate_one_epoch(leaf_model, val_dataloader)

In [ ]:
val_acc

In [ ]:
with torch.no_grad():
    logits_all = torch.zeros((val_dataloader.num_padded_samples, 5), dtype=float, device=leaf_model.device)
    labels_all = torch.zeros((val_dataloader.num_padded_samples), dtype=int, device=leaf_model.device)
    i = 0
    for imgs, labels in tqdm(val_dataloader):
        break

In [ ]:
with torch.no_grad():
    imgs = imgs.to(leaf_model.device)
    logits = leaf_model.model.forward(imgs)

In [ ]:
preds = logits.argmax(axis=-1)

In [ ]:
preds

In [ ]:
labels

In [ ]:
logits.shape

In [ ]:
imgs.shape